# 1.Limpieza y Transformación del Dataset de Calificaciones

Este notebook describe el proceso de limpieza y transformación del dataset de calificaciones.

El objetivo es garantizar que los datos sean **consistentes, estandarizados, anónimos y útiles** para el análisis posterior y la construcción de modelos de machine learning.

## 1.1. Definición de Variables - Dataset de Calificaciones

| Variable                  | Descripción                                                                 | Tipo de Dato         |
|---------------------------|-----------------------------------------------------------------------------|----------------------|
| `Sede`                    | Nombre de la sede o campus donde estudia el estudiante                     | `str` (texto)        |
| `Estudiante`              | Nombre completo del estudiante                                              | `str` (texto)        |
| `Documento de identidad`  | Identificador único del estudiante                                          | `str` (texto, hasheado) |
| `Grado`                   | Grado académico que cursa el estudiante (ej. 1, 2, 3...)                    | `int` (entero)       |
| `Grupo`                   | Letra o código del grupo del estudiante dentro del grado                   | `str` (texto)        |
| `Periodo`                 | Periodo académico (ej. I, II, III, IV)                                      | `str` (texto)        |
| `Año`                     | Año académico                                                               | `int` (entero)       |
| `Intensidad Horaria`      | Número de horas dedicadas a la asignatura                                   | `int` (entero)       |
| `Asignatura`              | Nombre de la materia evaluada (ej. Matemáticas, Inglés)                    | `str` (texto)        |
| `Cognitiva`               | Nota del componente cognitivo (conocimiento teórico)                        | `float` (decimal)    |
| `Procedimental`           | Nota del componente procedimental (habilidades prácticas)                   | `float` (decimal)    |
| `Actitudinal`             | Nota del componente actitudinal (comportamiento, participación)             | `float` (decimal)    |
| `Axiologica`              | Nota del componente axiológico (valores, ética)                             | `float` (decimal)    |
| `Docente`                 | Nombre del docente que reporta la calificación                              | `str` (texto)        |
| `Resultado`               | Nota final integrada de la asignatura                                       | `float` (decimal)    |
| `Nivel`                   | Nivel de desempeño asignado (ej. Bajo, Básico, Alto, Superior)              | `str` (texto)        |


In [1]:
import sys
import os
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

current_dir = os.path.dirname(os.path.abspath('__file__'))
project_root = os.path.abspath(os.path.join(current_dir, '..', '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from scripts.transform.data_cleaning_notes import NotesDataCleaner
from scripts.utils.hash_utility import HashUtility

## 1.2. Carga de dataset de Calificaciones

- Cargar fuente original de datos de calificaciones  
    - dataset de notas 2021-2022 limpio
    - dataset de notas 2023-2025 para limpiar


In [2]:
#hacer apend con las notas de los años anteriores
df_2021_2022 = pd.read_csv("../../data/raw/calificaciones-clean-2021-2022.csv")

df_2021_2022.info()
# Conteo de valores nulos
df_2021_2022.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16623 entries, 0 to 16622
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sede            16623 non-null  object 
 1   Año             16623 non-null  int64  
 2   Periodo         16623 non-null  int64  
 3   Grado           16623 non-null  int64  
 4   Identificación  16623 non-null  object 
 5   Estudiante      16623 non-null  object 
 6   Asignatura      16623 non-null  object 
 7   Cog             16579 non-null  float64
 8   Proc            16578 non-null  float64
 9   Axi             16575 non-null  float64
 10  Act             16576 non-null  float64
 11  Com             16576 non-null  float64
 12  Prom            16622 non-null  float64
dtypes: float64(6), int64(3), object(4)
memory usage: 1.6+ MB


Sede               0
Año                0
Periodo            0
Grado              0
Identificación     0
Estudiante         0
Asignatura         0
Cog               44
Proc              45
Axi               48
Act               47
Com               47
Prom               1
dtype: int64

In [3]:
df_2023_2025 = pd.read_csv("../../data/raw/calificaciones-raw-2023-2024-2025.csv")

df_2023_2025.info()
# Conteo de valores nulos
df_2023_2025.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32669 entries, 0 to 32668
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Sede                    32669 non-null  object
 1   Estudiante              32669 non-null  object
 2   Documento de identidad  32669 non-null  int64 
 3   Grado                   32669 non-null  int64 
 4   Grupo                   32669 non-null  object
 5   Periodo                 32669 non-null  object
 6   Año                     32669 non-null  int64 
 7   Intensidad Horaria      32669 non-null  int64 
 8   Asignatura              32669 non-null  object
 9   Cognitiva               32669 non-null  int64 
 10  Procedimental           32669 non-null  int64 
 11  Actitudinal             32669 non-null  int64 
 12  Axiologica              32669 non-null  int64 
 13  Docente                 32669 non-null  object
 14  Resultado               32669 non-null  int64 
 15  Ni

Sede                      0
Estudiante                0
Documento de identidad    0
Grado                     0
Grupo                     0
Periodo                   0
Año                       0
Intensidad Horaria        0
Asignatura                0
Cognitiva                 0
Procedimental             0
Actitudinal               0
Axiologica                0
Docente                   0
Resultado                 0
Nivel                     0
dtype: int64

## 1.3. Normalizar nombres de columnas

In [4]:
#imprimir nombres de columnas
print(df_2023_2025.columns)
print(df_2021_2022.columns)

Index(['Sede', 'Estudiante', 'Documento de identidad', 'Grado', 'Grupo',
       'Periodo', 'Año', 'Intensidad Horaria', 'Asignatura', 'Cognitiva',
       'Procedimental', 'Actitudinal', 'Axiologica', 'Docente', 'Resultado',
       'Nivel'],
      dtype='object')
Index(['Sede', 'Año', 'Periodo', 'Grado', 'Identificación', 'Estudiante',
       'Asignatura', 'Cog', 'Proc', 'Axi', 'Act', 'Com', 'Prom'],
      dtype='object')


In [5]:
#cambiar el nombre de la columna "id" a "id_calificacion"
df_2021_2022.rename(columns={"Prom": "Resultado"}, inplace=True)

df_2023_2025.rename(columns={"Cognitiva": "Cog"}, inplace=True)
df_2023_2025.rename(columns={"Procedimental": "Proc"}, inplace=True)
df_2023_2025.rename(columns={"Actitudinal": "Act"}, inplace=True)
df_2023_2025.rename(columns={"Axiologica": "Axi"}, inplace=True)


## 1.4. Limpieza de los datos
Realiza limpieza básica solo a df_2023_2025 ya que :ese proceso se corrio previamente para df_2021_2022
- Quita espacios de nombres de columnas  
- Convierte columnas de notas a numérico  
- Convierte texto a mayúsculas  

In [6]:
# comprobar que el metodo de limpieza existe
# y que se puede importar correctamente
from scripts.transform.data_cleaning_notes import NotesDataCleaner
print(dir(NotesDataCleaner))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'codificar_periodos', 'load_and_clean_data']


In [7]:
cleaner = NotesDataCleaner(df_2023_2025)
df_2023_2025 = cleaner.load_and_clean_data()
df_2023_2025.info()
df_2023_2025.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32669 entries, 0 to 32668
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Sede                    32669 non-null  object
 1   Estudiante              32669 non-null  object
 2   Documento_de_identidad  32669 non-null  int64 
 3   Grado                   32669 non-null  object
 4   Grupo                   32669 non-null  object
 5   Periodo                 32669 non-null  object
 6   Año                     32669 non-null  object
 7   Intensidad_Horaria      32669 non-null  int64 
 8   Asignatura              32669 non-null  object
 9   Cog                     32669 non-null  int64 
 10  Proc                    32669 non-null  int64 
 11  Act                     32669 non-null  int64 
 12  Axi                     32669 non-null  int64 
 13  Docente                 32669 non-null  object
 14  Resultado               32669 non-null  int64 
 15  Ni

Sede                      0
Estudiante                0
Documento_de_identidad    0
Grado                     0
Grupo                     0
Periodo                   0
Año                       0
Intensidad_Horaria        0
Asignatura                0
Cog                       0
Proc                      0
Act                       0
Axi                       0
Docente                   0
Resultado                 0
Nivel                     0
dtype: int64

## 1.5. Anonimización de los datos

Se aplicó **hashing seguro** a las variables sensibles:

- `id` se reemplaza con hashing 
- `Estudiante` (identificador único basado en nombre y sede) y aplica hashing

Estas columnas fueron eliminadas posteriormente para preservar la privacidad de los estudiantes.

- Se aplica funcion para codificar periodos

In [9]:
# Aplicar hashing al documento de identificación y eliminar la columna original
df_2023_2025["Identificación"] = df_2023_2025["Documento_de_identidad"].apply(HashUtility.hash_stable)
df_2023_2025.drop(columns=["Documento_de_identidad"], inplace=True)
# Aplicar hashing a los nombres y apellidos y eliminar la columna original
df_2023_2025["Estudiante"] = df_2023_2025["Estudiante"].replace(" ", "")+df_2023_2025["Sede"].astype(str)
df_2023_2025["Estudiante"] = df_2023_2025["Estudiante"].apply(HashUtility.hash_stable)


In [10]:
df_2023_2025= cleaner.codificar_periodos(df_2023_2025) 

## 1.6. Unificar todos los dataframes de notas

In [11]:
# Concatena los DataFrames
df_append = pd.concat([df_2023_2025, df_2021_2022], ignore_index=True)


In [12]:
df_append.info()
# Conteo de valores nulos
df_append.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49292 entries, 0 to 49291
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Sede                49292 non-null  object 
 1   Estudiante          49292 non-null  object 
 2   Grado               49292 non-null  object 
 3   Grupo               32669 non-null  object 
 4   Periodo             49292 non-null  object 
 5   Año                 49292 non-null  object 
 6   Intensidad_Horaria  32669 non-null  float64
 7   Asignatura          49292 non-null  object 
 8   Cog                 49248 non-null  float64
 9   Proc                49247 non-null  float64
 10  Act                 49245 non-null  float64
 11  Axi                 49244 non-null  float64
 12  Docente             32669 non-null  object 
 13  Resultado           49291 non-null  float64
 14  Nivel               32669 non-null  object 
 15  Identificación      49292 non-null  object 
 16  Com 

Sede                      0
Estudiante                0
Grado                     0
Grupo                 16623
Periodo                   0
Año                       0
Intensidad_Horaria    16623
Asignatura                0
Cog                      44
Proc                     45
Act                      47
Axi                      48
Docente               16623
Resultado                 1
Nivel                 16623
Identificación            0
Com                   32716
dtype: int64

In [14]:
##IMPRIMIR DUPLICADOS
duplicados = df_append[df_append.duplicated(subset=["Estudiante", "Identificación", "Grado", "Grupo", "Periodo", "Año", "Asignatura"], keep=False)]
df_append.drop_duplicates(subset=["Estudiante", "Identificación", "Grado", "Grupo", "Periodo", "Año", "Asignatura"], keep='first', inplace=True)
duplicados.info()


<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Sede                0 non-null      object 
 1   Estudiante          0 non-null      object 
 2   Grado               0 non-null      object 
 3   Grupo               0 non-null      object 
 4   Periodo             0 non-null      object 
 5   Año                 0 non-null      object 
 6   Intensidad_Horaria  0 non-null      float64
 7   Asignatura          0 non-null      object 
 8   Cog                 0 non-null      float64
 9   Proc                0 non-null      float64
 10  Act                 0 non-null      float64
 11  Axi                 0 non-null      float64
 12  Docente             0 non-null      object 
 13  Resultado           0 non-null      float64
 14  Nivel               0 non-null      object 
 15  Identificación      0 non-null      object 
 16  Com                 0 non

In [15]:
df_append.info()
# Conteo de valores nulos
df_append.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49292 entries, 0 to 49291
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Sede                49292 non-null  object 
 1   Estudiante          49292 non-null  object 
 2   Grado               49292 non-null  object 
 3   Grupo               32669 non-null  object 
 4   Periodo             49292 non-null  object 
 5   Año                 49292 non-null  object 
 6   Intensidad_Horaria  32669 non-null  float64
 7   Asignatura          49292 non-null  object 
 8   Cog                 49248 non-null  float64
 9   Proc                49247 non-null  float64
 10  Act                 49245 non-null  float64
 11  Axi                 49244 non-null  float64
 12  Docente             32669 non-null  object 
 13  Resultado           49291 non-null  float64
 14  Nivel               32669 non-null  object 
 15  Identificación      49292 non-null  object 
 16  Com 

Sede                      0
Estudiante                0
Grado                     0
Grupo                 16623
Periodo                   0
Año                       0
Intensidad_Horaria    16623
Asignatura                0
Cog                      44
Proc                     45
Act                      47
Axi                      48
Docente               16623
Resultado                 1
Nivel                 16623
Identificación            0
Com                   32716
dtype: int64

In [16]:
df_append.sample(5)

,Sede,Estudiante,Grado,Grupo,Periodo,Año,Intensidad_Horaria,Asignatura,Cog,Proc,Act,Axi,Docente,Resultado,Nivel,Identificación,Com
17536,FUSAGASUGÁ,90e1283eb582ed5619cabcd9ee332cffef82677032c0da...,2,A,2,2024,4.0,CIENCIAS NATURALES,91.0,94.0,94.0,100.0,ALEJANDRA LEÓN DICELIS,94.0,ALTO,365756628ae1424bc0862865e23ea33891575313cc45c4...,NaN
14867,FUSAGASUGÁ,da6b59d150b7a256b9575f8507186180d616d42191767d...,10,A,3,2024,1.0,LECTURA CRÍTICA,93.0,95.0,92.0,95.0,ARMANDO ÁVILA CONTRERAS,94.0,ALTO,ce66a313d6f71f78a8de5a85d19251743853b367dc8451...,NaN
10051,GIRARDOT,95e91145426ff51181bfb537a0ef6d1e74021feba3e502...,2,A,3,2023,3.0,EDUCACIÓN FÍSICA,94.0,94.0,93.0,94.0,FELIPE SÁNCHEZ SALDARRIAGA,94.0,ALTO,d50fbaaeb7e99a928cd8cb8ed22ecb7c80c9c3a07636d7...,NaN
18990,FUSAGASUGÁ,08ab0a29f140a4f6bd69ae927c3b0afce186253d9dc817...,9,A,2,2024,1.0,ARTES,79.0,79.0,80.0,80.0,XIOMARA LANZA TOLOSA,80.0,ALTO,095dc92b271953ffd13d7622841a9ed8cb9ae82ac6cca5...,NaN
18868,FUSAGASUGÁ,2b409f748e08152ac4629499b29d634afdd1714dd357b1...,8,A,2,2024,1.0,INNOVACIÓN Y EMPRENDIMIENTO,91.0,92.0,95.0,95.0,TATIANA RAMOS NOVA,92.0,ALTO,d42a967ec93c7782b43efd0a8dbc8dbd52457d0e8fc7f8...,NaN


In [17]:
#reemplace nombres de valores de asignaturas
df_append["Asignatura"] = df_append["Asignatura"].replace({
    "ENGLISH": "INGLÉS",
    "FRANÇAIS": "FRANCÉS"})


In [18]:
df_append.to_csv("../../data/processed/data_clean_notes_2021_2025.csv", index=False)